In [1]:
import json
import pandas as pd
import sqlalchemy

receipts = []
with open('data/receipts.json', 'r') as file:
    for line in file:
        receipts.append(json.loads(line))
df = pd.json_normalize(receipts)
df.shape

(1119, 15)

In [2]:
# Creating a new DF
df_receipt = df.copy()

In [3]:
# Change data types and rename columns
df_receipt = df_receipt.astype({'pointsEarned': 'float64'})
df_receipt = df_receipt.astype({'totalSpent': 'float64'})
df_receipt['createDate'] = pd.to_datetime(df_receipt['createDate.$date'], unit='ms')
df_receipt['dateScanned'] = pd.to_datetime(df_receipt['dateScanned.$date'], unit='ms')
df_receipt['finishedDate'] = pd.to_datetime(df_receipt['finishedDate.$date'], unit='ms')
df_receipt['modifyDate'] = pd.to_datetime(df_receipt['modifyDate.$date'], unit='ms')
df_receipt['pointsAwardedDate'] = pd.to_datetime(df_receipt['pointsAwardedDate.$date'], unit='ms')
df_receipt['purchaseDate'] = pd.to_datetime(df_receipt['purchaseDate.$date'], unit='ms')
df_receipt.rename(columns={'_id.$oid':'receipt_id'},inplace=True)
df_receipt.shape

(1119, 21)

In [4]:
# Receipt Status DF
df_receipt_status = df_receipt[['receipt_id','rewardsReceiptStatus']]
df_receipt_status.head()

,receipt_id,rewardsReceiptStatus
0,5ff1e1eb0a720f0523000575,FINISHED
1,5ff1e1bb0a720f052300056b,FINISHED
2,5ff1e1f10a720f052300057a,REJECTED
3,5ff1e1ee0a7214ada100056f,FINISHED
4,5ff1e1d20a7214ada1000561,FINISHED


In [5]:
# Receipt Bonus Points Earned Reason DF
df_bp_reason = df_receipt[['receipt_id','bonusPointsEarnedReason']]
df_bp_reason.head()

,receipt_id,bonusPointsEarnedReason
0,5ff1e1eb0a720f0523000575,"Receipt number 2 completed, bonus point schedu..."
1,5ff1e1bb0a720f052300056b,"Receipt number 5 completed, bonus point schedu..."
2,5ff1e1f10a720f052300057a,All-receipts receipt bonus
3,5ff1e1ee0a7214ada100056f,All-receipts receipt bonus
4,5ff1e1d20a7214ada1000561,All-receipts receipt bonus


In [6]:
# Note: Receipts that don't have the required key or did not buy any item are not considered
data = {}
data['result'] = [receipts[x] for x in range(len(receipts)) if receipts[x].get('rewardsReceiptItemList')]
df_receipt_item = pd.json_normalize(data['result'],'rewardsReceiptItemList', meta=['_id'])
df_receipt_item['receipt_id'] = df_receipt_item['_id'].apply(pd.Series)
df_receipt_item.drop('_id', axis=1, inplace=True)
df_receipt_item.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId,receipt_id
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eb0a720f0523000575
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1f10a720f052300057a
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1ee0a7214ada100056f


In [7]:
# drop columns from df_receipt
df_receipt.drop(['createDate.$date','dateScanned.$date','finishedDate.$date','modifyDate.$date','pointsAwardedDate.$date','purchaseDate.$date','rewardsReceiptItemList','rewardsReceiptStatus','bonusPointsEarnedReason'], axis=1, inplace=True)
df_receipt.head()

,bonusPointsEarned,pointsEarned,purchasedItemCount,totalSpent,userId,receipt_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,500.0,5.0,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,150.0,2.0,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,5.0,1.0,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,5.0,4.0,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,5.0,2.0,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


Brands Data

In [8]:
brands = []
with open('data/brands.json', 'r') as file:
    for line in file:
        brands.append(json.loads(line))
        
df_brands = pd.json_normalize(brands)
df_brands.rename(columns={'_id.$oid':'brand_id','cpg.$id.$oid':'cpg','cpg.$ref':'ref'}, inplace=True)
df_brands.head()

,barcode,category,categoryCode,name,topBrand,brand_id,cpg,ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


Users Data

In [9]:
# Check for duplicate ID in USers table
users = []
with open('data/users.json', 'r') as file:
    for line in file:
        users.append(json.loads(line))
df_users = pd.json_normalize(users)
df_users['createdDate'] = pd.to_datetime(df_users['createdDate.$date'],unit='ms')
df_users['lastLogin'] = pd.to_datetime(df_users['lastLogin.$date'],unit='ms')
df_users.rename(columns={'_id.$oid':'user_id'}, inplace=True)
df_users.drop(['createdDate.$date','lastLogin.$date'], axis=1,inplace=True)
df_users.head()

,active,role,signUpSource,state,user_id,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872


# Create SQLite DB and Insert data

In [10]:
engine = sqlalchemy.create_engine('sqlite:///fetch_rewards_exc.db', echo = True) 

In [11]:
df_receipt.to_sql('receipts_fact', con=engine, if_exists='replace')
df_receipt_item.to_sql('item_list_dim', con=engine, if_exists='replace')
df_brands.to_sql('brands_dim', con=engine, if_exists='replace')
df_users.to_sql('users_dim', con=engine, if_exists='replace')
df_receipt_status.to_sql('receipt_status_dim', con=engine, if_exists='replace')
df_bp_reason.to_sql('bp_reason_dim', con=engine, if_exists='replace')

2024-12-17 14:27:43,141 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-17 14:27:43,154 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("receipts_fact")
2024-12-17 14:27:43,156 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-17 14:27:43,158 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("receipts_fact")
2024-12-17 14:27:43,159 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-17 14:27:43,163 INFO sqlalchemy.engine.Engine 
CREATE TABLE receipts_fact (
	"index" BIGINT, 
	"bonusPointsEarned" FLOAT, 
	"pointsEarned" FLOAT, 
	"purchasedItemCount" FLOAT, 
	"totalSpent" FLOAT, 
	"userId" TEXT, 
	receipt_id TEXT, 
	"createDate" DATETIME, 
	"dateScanned" DATETIME, 
	"finishedDate" DATETIME, 
	"modifyDate" DATETIME, 
	"pointsAwardedDate" DATETIME, 
	"purchaseDate" DATETIME
)


2024-12-17 14:27:43,164 INFO sqlalchemy.engine.Engine [no key 0.00112s] ()
2024-12-17 14:27:43,168 INFO sqlalchemy.engine.Engine CREATE INDEX ix_receipts_fact_index ON receipts_fact ("index")
2024-12-

1119

Connect to DB

In [12]:
%load_ext sql
%sql sqlite:///fetch_rewards_exc.db

Connecting to 'sqlite:///fetch_rewards_exc.db'

# SQL Queries

Query 1: What are the top 5 brands by receipts scanned for most recent month?

Note: Both queries that provide Top 5 'TopBrand' and Top 5 Brands that were bought most frequently are included.

In [13]:
%%sql
with cte_brands as (select barcode, name from brands_dim where topBrand is True group by 1,2) -- 'Top Brand' is True

select name, count(distinct receipt_id) as receipt_count
from item_list_dim d
join cte_brands b on d.barcode = b.barcode
where receipt_id in (select receipt_id from receipts_fact where strftime('%Y-%m', date(dateScanned)) in (select strftime('%Y-%m', max(date(dateScanned))) from receipts_fact) group by 1)
group by 1
order by 2 desc
limit 5;

Running query in 'sqlite:///fetch_rewards_exc.db'

name,receipt_count


In [14]:
%%sql
with cte_top as (
    select barcode, count(distinct receipt_id) as receipt_count
    from item_list_dim d
    where receipt_id in (select receipt_id from receipts_fact where strftime('%Y-%m', date(dateScanned)) in (select strftime('%Y-%m', max(date(dateScanned))) from receipts_fact) group by 1)
    group by 1
    order by 2 desc
    limit 5
)

select t.barcode, name, receipt_count
from cte_top t
left join brands_dim b on t.barcode = b.barcode;

Running query in 'sqlite:///fetch_rewards_exc.db'

barcode,name,receipt_count
B07BRRLSVC,None,13
B076FJ92M4,None,13


Query 2: How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

Note: Both queries that provide Top 5 'TopBrand' and Top 5 Brands that are bought most frequently are included.

In [15]:
%%sql
with cte_months0 as (select strftime('%Y-%m', date(dateScanned)) as months from receipts_fact group by 1),

cte_months as (select *, row_number() over(order by months desc) as rn from cte_months0),

cte_brands as (select barcode, name from brands_dim where topBrand is True group by 1,2), -- 'Top Brand' is True

cte_recent as (select 'recent_month' as month_label, name, count(distinct receipt_id) as receipt_count
from item_list_dim d
join cte_brands b on d.barcode = b.barcode
where receipt_id in (select receipt_id from receipts_fact where strftime('%Y-%m', date(dateScanned)) in (select months from cte_months where rn = 1) group by 1)
group by 1,2
order by 3 desc
limit 5),

cte_previous as (select 'previous_month' as month_label, name, count(distinct receipt_id) as receipt_count
from item_list_dim d
join cte_brands b on d.barcode = b.barcode
where receipt_id in (select receipt_id from receipts_fact where strftime('%Y-%m', date(dateScanned)) in (select months from cte_months where rn = 2) group by 1)
group by 1,2
order by 3 desc
limit 5)

select * from cte_recent
union all
select * from cte_previous;

Running query in 'sqlite:///fetch_rewards_exc.db'

month_label,name,receipt_count


In [16]:
%%sql
with cte_months0 as (select strftime('%Y-%m', date(dateScanned)) as months from receipts_fact group by 1),

cte_months as (select *, row_number() over(order by months desc) as rn from cte_months0),

cte_recent as (select 'recent_month' as month_label, d.barcode, name, count(distinct receipt_id) as receipt_count
from item_list_dim d
left join brands_dim b on d.barcode = b.barcode
where receipt_id in (select receipt_id from receipts_fact where strftime('%Y-%m', date(dateScanned)) in (select months from cte_months where rn = 1) group by 1)
group by 1,2,3
order by 4 desc
limit 5),

cte_previous as (select 'previous_month' as month_label, d.barcode, name, count(distinct receipt_id) as receipt_count
from item_list_dim d
left join brands_dim b on d.barcode = b.barcode
where receipt_id in (select receipt_id from receipts_fact where strftime('%Y-%m', date(dateScanned)) in (select months from cte_months where rn = 2) group by 1)
group by 1,2,3
order by 4 desc
limit 5)

select * from cte_recent
union all
select * from cte_previous;

Running query in 'sqlite:///fetch_rewards_exc.db'

month_label,barcode,name,receipt_count
recent_month,B076FJ92M4,None,13
recent_month,B07BRRLSVC,None,13
previous_month,None,None,36
previous_month,4011,None,33
previous_month,B076FJ92M4,None,27
previous_month,B07BRRLSVC,None,27
previous_month,1234,None,5


Query 3: When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [17]:
%%sql
with cte_accepted as (
    select 1 as key, coalesce(round(avg(totalSpent),2),0) as avg_spend 
    from receipts_fact 
    where receipt_id in (select receipt_id from receipt_status_dim where rewardsReceiptStatus = 'FINISHED' group by 1)
    ),

cte_rejected as (
    select 1 as key, coalesce(round(avg(totalSpent),2),0) as avg_spend 
    from receipts_fact 
    where receipt_id in (select receipt_id from receipt_status_dim where rewardsReceiptStatus = 'REJECTED' group by 1)
    )

select 
    case 
        when a.avg_spend > r.avg_spend then 'Accepted' 
        when a.avg_spend < r.avg_spend then 'Rejected'
        else 'both same' end as receipt_status,
    case 
        when a.avg_spend > r.avg_spend then a.avg_spend
        when a.avg_spend < r.avg_spend then r.avg_spend
        else a.avg_spend end as avg_spend
from cte_accepted a 
join cte_rejected r on a.key = r.key

Running query in 'sqlite:///fetch_rewards_exc.db'

receipt_status,avg_spend
Accepted,80.85


Query 4: When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [18]:
%%sql
with cte_accepted as (
    select 1 as key, cast(coalesce(round(sum(purchasedItemCount),2),0) as int) as item_count 
    from receipts_fact 
    where receipt_id in (select receipt_id from receipt_status_dim where rewardsReceiptStatus = 'FINISHED' group by 1)
    ),

cte_rejected as (
    select 1 as key, cast(coalesce(round(sum(purchasedItemCount),2),0) as int) as item_count 
    from receipts_fact 
    where receipt_id in (select receipt_id from receipt_status_dim where rewardsReceiptStatus = 'REJECTED' group by 1)
    )

select 
    case 
        when a.item_count > r.item_count then 'Accepted' 
        when a.item_count < r.item_count then 'Rejected'
        else 'both same' end as receipt_status,
    case 
        when a.item_count > r.item_count then a.item_count
        when a.item_count < r.item_count then r.item_count
        else a.item_count end as item_purchased
from cte_accepted a 
join cte_rejected r on a.key = r.key

Running query in 'sqlite:///fetch_rewards_exc.db'

receipt_status,item_purchased
Accepted,8184


Query 5: Which brand has the most spend among users who were created within the past 6 months? \
Note: \
(1) Date of the last receipt scanned is considered as the most recent date \
(2) Inner Join used to display Brands/Barcodes available in both tables

In [19]:
%%sql
with cte_range as (select max(date(dateScanned)) as max_date, date(max(date(dateScanned)),'-6 month') as min_date from receipts_fact),

cte_receipt as (
select receipt_id from receipts_fact 
where userId in (select distinct user_id from users_dim where date(createdDate) >= (select min_date from cte_range) and date(createdDate) <= (select max_date from cte_range))
group by 1
)

select l.barcode, name, round(sum(finalPrice),2) as total_spend
from item_list_dim l
join brands_dim b on l.barcode = b.barcode
where receipt_id in (select * from cte_receipt)
group by 1,2
order by 3 desc
limit 1;

Running query in 'sqlite:///fetch_rewards_exc.db'

barcode,name,total_spend
511111802358,Cracker Barrel Cheese,196.98


Query 6: Which brand has the most transactions among users who were created within the past 6 months? \
Note: \
(1) Date of the last receipt scanned is considered as the most recent date \
(2) Inner Join used to display Brands/Barcodes available in both tables

In [20]:
%%sql
with cte_range as (select max(date(dateScanned)) as max_date, date(max(date(dateScanned)),'-6 month') as min_date from receipts_fact),

cte_receipt as (
select receipt_id from receipts_fact 
where userId in (select distinct user_id from users_dim where date(createdDate) >= (select min_date from cte_range) and date(createdDate) <= (select max_date from cte_range))
group by 1
)

select l.barcode, name, count(distinct receipt_id) as transaction_count
from item_list_dim l
join brands_dim b on l.barcode = b.barcode
where receipt_id in (select * from cte_receipt)
group by 1,2
order by 3 desc
limit 1;

Running query in 'sqlite:///fetch_rewards_exc.db'

barcode,name,transaction_count
511111001485,Tostitos,11


## Data Quality Issues

1. Incomplete data: Out of 1119 receipts, only 679 receipts have purchased items and the remaining are NULL

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    object 
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 non-null   object 
 6   totalSpent               684 non-null    object 
 7   userId                   1119 non-null   object 
 8   _id.$oid                 1119 non-null   object 
 9   createDate.$date         1119 non-null   int64  
 10  dateScanned.$date        1119 non-null   int64  
 11  finishedDate.$date       568 non-null    float64
 12  modifyDate.$date         1119 non-null   int64  
 13  pointsAwardedDate.$date  537 non-null    float64
 14  purchaseDate.$date      

2. Incorrect data type: Incorrect data type identified for columns 'totalSpent', 'pointsEarned', 'createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', and 'purchaseDate'

In [22]:
df.dtypes

bonusPointsEarned          float64
bonusPointsEarnedReason     object
pointsEarned                object
purchasedItemCount         float64
rewardsReceiptItemList      object
rewardsReceiptStatus        object
totalSpent                  object
userId                      object
_id.$oid                    object
createDate.$date             int64
dateScanned.$date            int64
finishedDate.$date         float64
modifyDate.$date             int64
pointsAwardedDate.$date    float64
purchaseDate.$date         float64
dtype: object

3. Invalid Data: As shown below, Max(purchaseDate) is greater than Max(dateScanned) and the SQL below displays the incorrect record

In [23]:
df_receipt.describe().loc['max']

bonusPointsEarned                          750.0
pointsEarned                             10199.8
purchasedItemCount                         689.0
totalSpent                               4721.95
createDate            2021-03-01 23:17:34.772000
dateScanned           2021-03-01 23:17:34.772000
finishedDate                 2021-02-26 22:36:25
modifyDate            2021-03-01 23:17:34.772000
pointsAwardedDate            2021-02-26 22:36:25
purchaseDate                 2021-03-08 17:37:13
Name: max, dtype: object

In [24]:
%%sql
select *
from receipts_fact
where date(purchaseDate) in (select date(max(purchaseDate)) from receipts_fact)

Running query in 'sqlite:///fetch_rewards_exc.db'

index,bonusPointsEarned,pointsEarned,purchasedItemCount,totalSpent,userId,receipt_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
871,25.0,25.0,1.0,1.0,6021768799409b11fcf8987f,602176c90a7214d8e9000028,2021-02-08 17:37:13.000000,2021-02-08 17:37:13.000000,2021-02-08 17:37:13.000000,2021-02-08 17:37:13.000000,2021-02-08 17:37:13.000000,2021-03-08 17:37:13.000000


4. Data Inconsistency: Points awarded to receipts that were not 'Accepted'

In [25]:
%%sql
select count(distinct receipt_id) as receipt_count
from receipts_fact
where receipt_id in (select receipt_id from receipt_status_dim where rewardsReceiptStatus <> 'FINISHED') and pointsEarned is not null

Running query in 'sqlite:///fetch_rewards_exc.db'

receipt_count
91


5. Data Inconsistency: Points not awarded to receipts that were 'Accepted'

In [26]:
%%sql
with cte as (
select distinct receipt_id
from receipts_fact
where pointsEarned < 1 and receipt_id in (select receipt_id from receipt_status_dim where rewardsReceiptStatus = 'FINISHED'))

select c.receipt_id, bonusPointsEarnedReason -- count(c.receipt_id)
from cte c
join bp_reason_dim b
on c.receipt_id = b.receipt_id

Running query in 'sqlite:///fetch_rewards_exc.db'

receipt_id,bonusPointsEarnedReason
5ff774200a7214ada1000638,None
5ff8c8880a7214adca000008,None
5ffc8cc20a720f05c5000026,None
5ffc8cb40a720f05c5000025,None


6. Missing Data: Count of Barcode/Brand not available in Brand dataset

In [27]:
%%sql
select count(distinct barcode) as brands_missing
from item_list_dim
where barcode not in (select distinct barcode from brands_dim)

Running query in 'sqlite:///fetch_rewards_exc.db'

brands_missing
552


7. Data Redundancy: Duplicate records identified in User dataset

In [28]:
%%sql
select active, role, signUpSource, state, user_id, createdDate, lastLogin, count(*) as duplicate_records
from users_dim
group by 1,2,3,4,5,6,7
having count(*) > 1;

Running query in 'sqlite:///fetch_rewards_exc.db'

active,role,signUpSource,state,user_id,createdDate,lastLogin,duplicate_records
1,consumer,None,None,5a43c08fe4b014fd6b6a0612,2017-12-27 15:47:27.059000,2021-02-12 16:22:37.155000,8
1,consumer,Email,None,60145ff384231211ce796d51,2021-01-29 19:20:19.722000,None,2
1,consumer,Email,None,60186237c8b50e11d8454d5f,2021-02-01 20:19:03.551000,None,5
1,consumer,Email,AL,5ffe115404929101d0aaebb2,2021-01-12 21:15:00.208000,None,2
1,consumer,Email,AL,6000b75bbe5fc96dfee1d4d3,2021-01-14 21:27:55.938000,None,6
1,consumer,Email,OH,60147d2ac8b50e11d8453f53,2021-01-29 21:24:58.184000,None,5
1,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800000,2021-01-03 15:25:37.857999,11
1,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554000,2021-01-03 15:25:30.596999,4
1,consumer,Email,WI,5ff36a3862fde912123a4460,2021-01-04 19:19:20.352000,2021-01-04 19:23:05.340000,5
1,consumer,Email,WI,5ff36be7135e7011bcb856d3,2021-01-04 19:26:31.239000,2021-01-04 19:29:52.726000,3


8. Data Redundancy: One to one mapping identified between Barcode and Brand ID for most records, however, the following 7 Barcodes have multiple Brand ID 

In [29]:
%%sql
select barcode, count(distinct brand_id)
from brands_dim
group by 1
having count(distinct brand_id) > 1

Running query in 'sqlite:///fetch_rewards_exc.db'

barcode,count(distinct brand_id)
511111004790,2
511111204923,2
511111305125,2
511111504139,2
511111504788,2
511111605058,2
511111704140,2
